In [ ]:
import pandas as pd
import lichess.api
from lichess.format import SINGLE_PGN
import altair as alt
from sklearn.linear_model import LinearRegression, LogisticRegression

In [ ]:
USERNAME = "unwrap"
SPEED = "blitz"
VARIANT = "standard"

In [ ]:
def create_game_stats(games):
    rating = []
    timestamp = []
    status = []
    color = []
    winner = []
    all_stats = set()
    for game in games:
        if game["speed"] == SPEED and game["rated"] == True and game["variant"] == VARIANT:
            player = game["players"]["white"]["user"]["name"]
            if player == USERNAME:
                rating.append(game["players"]["white"]["rating"])
                color.append("white")
            else:
                rating.append(game["players"]["black"]["rating"])
                color.append("black")
            timestamp.append(game["createdAt"])
            game_status = game["status"] 
            all_stats.add(game_status)
            status.append(game_status)
            if game_status == "outoftime":
                winner.append(color[-1])
            elif game_status != "draw" and game_status != "stalemate" and game_status != "timeout":
                try:
                    winner.append(game["winner"])
                except:
                    print(game)
            else:
                winner.append(None)

    df = pd.DataFrame(dict(rating=rating, timestamp=timestamp, status=status, color=color, winner=winner))

In [ ]:
def enrich_df(df):
    df["win"] = df["color"] == df["winner"]
    df['date'] =  pd.to_datetime(df['timestamp'], unit="ms")
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    game_count_df = df.groupby(["year", "month", "day"]).count().reset_index()[["year", "month", "day", "rating",]].rename(columns={"rating": "game_count"})
    win_count_df  = df.groupby(["year", "month", "day"]).sum().reset_index()[["year", "month", "day", "win",]].rename(columns={"win": "total_wins"})
    df = pd.merge(df, game_count_df, on=["year", "month", "day"], how="left")
    df = pd.merge(df, win_count_df, on=["year", "month", "day"], how="left")
    return df

In [ ]:
def predict_rating(df):
    Y = df.iloc[:, 0].values.reshape(-1, 1)
    X = df.iloc[:, 1].values.reshape(-1, 1)
    linear_regressor = LinearRegression() # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions
    print(f"Linear Regression R^2: {linear_regressor.score(X,Y)}")
    df["rating_prediction"] = Y_pred
    return df

In [ ]:
games = list(lichess.api.user_games(USERNAME, max=10000))

In [ ]:
df = create_game_stats(games)
df = df.iloc[:-20] # Remove the first 20 games
df = enrich_df(df)
df = predict_rating(df)
df.tail()

In [ ]:
chart = alt.Chart(df).mark_area(
    color="lightblue",
    interpolate='step-after',
    line=True
).encode(x='timestamp:T', y='rating').mark_area(opacity=0.6)

chart_line = alt.Chart(df).mark_line(
    color="red",
    line=True
).encode(x='timestamp:T',y="rating_prediction")

In [ ]:
game_count_chart = alt.Chart(df).mark_bar().encode(
    x="date",
    y="game_count",
    color=alt.value("#DC143C"),
).properties(width=1200, height=100)
win_count_chart = alt.Chart(df).mark_bar().encode(
    x="date",
    y="total_wins",
    color=alt.value("#228B22"),
).properties(width=1200, height=100)

In [ ]:
out = chart + chart_line
out.properties(width=1200, height=400) & game_count_chart + win_count_chart

In [ ]:
df